An attempt to move scanID folders to correct sessionID folders, then rename the files inside them to match:
"sub-01_ses-{sessionID}_run-{repeatID}_T1w.nii

In [1]:
# Load libraries
import pandas as pd
from pathlib import Path
from math import isnan
import os
import json

In [2]:
ROOT_PATH = "/storage/research/cinn_comp/ThalSR/zan/derivatives/test/practice"
MODALITY = "MR"
BODY_PART_EXAMINED = "HEAD"
MR_ACQUISIITON_TYPE = "3D"

Create a dataframe object from the excel file.

In [4]:
# Load FTHP_Metadata excel doc as a pandas dataframe
df = pd.read_excel("FTHP_Metadata.xlsx", sheet_name='data')
df.head(5)

,scanID,included into thalamus study (see EuroRad paper Opfer et al. 2022),sessionID,repeatID,scannerID,sequenceIDscanner,siteID,exclude,excludeReason,numFiles,...,orientationCode,rows,columns,repetitionTime,echoTime,sliceThickness,spacingBetweenSlices,flipAngle,pixelSpacing1,pixelSpacing2
0,0,1,146,2,116,1,71,0,NaN,142,...,2,352,352,7.4497,3.541,1.2,1.2,8.0,0.727,0.727
1,1,1,96,2,78,1,58,0,NaN,160,...,2,256,256,2070.0000,3.220,1.0,NaN,15.0,0.977,0.977
2,2,1,61,1,52,1,84,0,NaN,144,...,2,256,208,980.0000,2.950,1.0,NaN,15.0,0.977,0.977
3,3,1,47,0,39,1,95,0,NaN,150,...,3,256,256,7.3164,3.336,1.1,1.1,8.0,0.944,0.944
4,4,1,17,3,14,1,91,0,NaN,160,...,2,256,256,2200.0000,2.680,1.0,NaN,8.0,0.977,0.977


The below chunk renames folders/files if they are in the same directory.

In [7]:
# Create a for loop that iterates across the range of values in "sessionID"
for index,row in df.iterrows():
    # Define variables
    scan_id = row["scanID"]
    session_id = row["sessionID"]
    repeat_id = row["repeatID"]
    # Define path that contains the files currently, and the path to which we want to move files to, which also renames the files in BIDS format
    from_path = Path(f"/storage/research/cinn_comp/ThalSR/zan/derivatives/test/practice/scanID/{scan_id}/{scan_id}.nii")
    to_path = Path(f"/storage/research/cinn_comp/ThalSR/zan/derivatives/test/practice/ses-{session_id}/anat/sub-01_ses-{session_id}_run-{repeat_id+1}_T1w.nii")
    # Check for errors instead of just stopping the program
    if not os.path.exists(from_path):
        print(f"{from_path} does not exist!")
        continue
    # Move files from and to defined locations, renaming them in the process
    os.rename(from_path, to_path)

    # If one of these variables is NaN, we skip it for now!!!
    slice_thickness = row["sliceThickness"]
    echo_time = row["echoTime"]
    rep_time = row["repetitionTime"]
    flip_angle = row["flipAngle"]
    if isnan(slice_thickness) or isnan(echo_time) or isnan(rep_time) or isnan(flip_angle):
        continue

    to_json = Path(f"/storage/research/cinn_comp/ThalSR/zan/derivatives/test/practice/ses-{session_id}/anat/sub-01_ses-{session_id}_run-{repeat_id+1}_T1w.json")
    current_obj = {
        "Modality": MODALITY,
        "MagneticFieldStrength": row["fieldStrength"],
        "Manufacturer": row["manufacturer"],
        "ManufacturersModelName": row["model"],
        "BodyPartExamined": BODY_PART_EXAMINED,
        "MRAcquisitionType": MR_ACQUISIITON_TYPE,
        "SeriesDescription": row["seriesDescription"],
        "ProtocolName": row["protocolName"],
        "SliceThickness": slice_thickness,
        "EchoTime": echo_time,
        "RepetitionTime": rep_time,
        "FlipAngle": flip_angle
        # "BaseResolution": 224,
        # "PhaseEncodingSteps": 186,
        # "AcquisitionMatrixPE": 186,
        # "ReconMatrixPE": 186
    }
    with open(to_json, "w", encoding="utf-8") as file:
        json.dump(current_obj, file, ensure_ascii=False, indent=4)




FileNotFoundError: [Errno 2] No such file or directory: '/storage/research/cinn_comp/ThalSR/zan/derivatives/test/practice/scanID/0/0.nii' -> '/storage/research/cinn_comp/ThalSR/zan/derivatives/test/practice/ses-146/anat/sub-01_ses-146_run-3_T1w.nii'